# CLI automation with Cisco pyATS 🚀

**pyATS** (Python Automated Test System) is a Cisco-developed Python framework for **network test automation and verification**. It helps network engineers and developers **automate configuration, testing, and validation** on real and virtual devices.

---

### 🧩 Components

- **pyATS Core** 🔹  
  The foundation for network automation, managing devices, connections, and test execution.

- **Genie** 🧪  
  Cisco’s parsing and abstraction library. Converts raw device output into structured data (facts) and provides configuration models.

- **Unicon** 🔌  
  Handles **device connectivity** (SSH, Telnet, console) and session management.

- **Testbeds** 🗺️  
  YAML-based definitions of devices, connections, credentials, and topology for consistent automation.

- **Ops and Conf Libraries** ⚙️  
  - **Ops**: Learn device state, interfaces, routing tables, etc.  
  - **Conf**: Build or rollback configurations programmatically.

---

### 🌟 Benefits

- ✅ Automate repetitive network tasks  
- ✅ Test configurations and verify device state  
- ✅ Generate dry-run config previews and diffs  
- ✅ Multi-vendor and multi-platform support (with Genie)  
- ✅ Safe rollback and candidate configuration support on IOS-XR and other platforms  

---

🔁 First of all, create a virtual environment with the following commands:

In [ ]:
!python3 -m venv .venv && source .venv/bin/activate
!pip install -r requirements.txt

### 1️⃣ Create and validate my testbed
Now, let's create a `topology.yaml` file where we specify which are all our devices, and how to connect to them.</br>
We then proceed to validate that file, and we can even run a basic connectivity attempt on each of the included devices. 

In [74]:
!pyats validate testbed pyATS/testbed.yaml --connect

Loading testbed file: pyATS/testbed.yaml
--------------------------------------------------------------------------------

Testbed Name:
    NetworkAutoDemo

Testbed Devices:
.
`-- sandbox1 [iosxr/ASR9K]

YAML Lint Messages
------------------

Connection Check
----------------

    note that connection checks are not 100% accurate - it does not take
    into account that connection implementations may choose to interpret
    the entire connection block differently.

    For example - Unicon autouses A/B console/standby, but does not allow
    explicit connection to B.

 - sandbox1/cli             [PASSED]                                          

Warning Messages
----------------
 - Device 'sandbox1' has no interface definitions



### 2️⃣ Connect to my device and retrieve raw/parsed configs
Now, let's roll our sleeves a little bit and get hands-on coding!</br>
Let's connect to our device using the info in our `testbed.yaml` file.</br>
pyATS will use the `unicon` library and the information from the yaml file to attempt to connect to the device `sandbox1`.

In [12]:
import json
from pyats.topology import loader

In [75]:
testbed = loader.load('pyATS/testbed.yaml')
device = testbed.devices['sandbox1']

device.connect()
print("✅ Connected successfully to", device.name)


2025-11-12 10:53:28,492: %UNICON-INFO: +++ sandbox1 logfile sandbox1-cli-1762944808.log +++

2025-11-12 10:53:28,492: %UNICON-INFO: +++ Unicon plugin iosxr (unicon.plugins.iosxr) +++

Hello there! Hoping you are having a great day
... Welcome to 'ios',
your favorite CISCO.IOSXR.IOSXR Sandbox



2025-11-12 10:53:30,054: %UNICON-INFO: +++ connection to spawn: ssh -l admin 131.226.217.150 -p 22, id: 4679754576 +++

2025-11-12 10:53:30,055: %UNICON-INFO: connection to sandbox1
(admin@131.226.217.150) Password: 


RP/0/RP0/CPU0:ios#

2025-11-12 10:53:31,000: %UNICON-INFO: Storing credentials from default as current_credentials

2025-11-12 10:53:31,002: %UNICON-WARNING: Invalid hostname detected: found ios, expected sandbox1

2025-11-12 10:53:31,003: %UNICON-INFO: +++ initializing handle +++

2025-11-12 10:53:31,192: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'terminal length 0' +++
terminal length 0
Wed Nov 12 11:40:56.384 UTC
RP/0/RP0/CPU0:ios#

2025-11-12 10:53:31,736: 

🔌 Lovely. Now, we want to get **information about the interfaces in this device**.</br>
For this, let's use the good-old `show ip interfaces brief`.

In [8]:
device.execute("show ip interface brief")


2025-11-12 09:32:43,037: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'show ip interface brief' +++
show ip interface brief
Wed Nov 12 10:20:08.849 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
Loopback100                    1.1.1.100       Up              Up       default 
Loopback555                    unassigned      Up              Up       default 
MgmtEth0/RP0/CPU0/0            10.10.20.175    Up              Up       default 
GigabitEthernet0/0/0/0         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/1         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/2         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/3         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/4         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/5         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0

'\rWed Nov 12 10:20:08.849 UTC\r\n\r\nInterface                      IP-Address      Status          Protocol Vrf-Name\r\nLoopback100                    1.1.1.100       Up              Up       default \r\nLoopback555                    unassigned      Up              Up       default \r\nMgmtEth0/RP0/CPU0/0            10.10.20.175    Up              Up       default \r\nGigabitEthernet0/0/0/0         unassigned      Shutdown        Down     default \r\nGigabitEthernet0/0/0/1         unassigned      Shutdown        Down     default \r\nGigabitEthernet0/0/0/2         unassigned      Shutdown        Down     default \r\nGigabitEthernet0/0/0/3         unassigned      Shutdown        Down     default \r\nGigabitEthernet0/0/0/4         unassigned      Shutdown        Down     default \r\nGigabitEthernet0/0/0/5         unassigned      Shutdown        Down     default \r\nGigabitEthernet0/0/0/6         unassigned      Shutdown        Down     default'

🔀 Beautiful. But let's imagine that I want to work with this info and get the exact names and IP addresses only. It's gonna be a bit of a regex or TextFMS burden, isn't it?</br></br>
**Let's use [the built-in Genie parses instead! 🧞](https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/parsers)**

In [76]:
genie_parsed_interfaces = device.parse("show ip interface brief")
print(json.dumps(genie_parsed_interfaces,indent=4))


2025-11-12 10:57:51,089: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'show ip interface brief' +++
show ip interface brief
Wed Nov 12 11:45:16.272 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
Loopback100                    1.1.1.100       Up              Up       default 
Loopback200                    unassigned      Up              Up       default 
Loopback300                    unassigned      Up              Up       default 
Loopback555                    unassigned      Up              Up       default 
MgmtEth0/RP0/CPU0/0            10.10.20.175    Up              Up       default 
GigabitEthernet0/0/0/0         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/1         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/2         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/3         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0

🧞 That's better, isn't it? A beautiful Python dictionary to work with ...

### 🧠 Config, learn and compare

pyATS comes bundled with a powerful feature called `learn`. It allows us to take snapshots of a specific configuration of a device to later use them to compare changes.
This is **[a series of CLI commands related to a specific feature](https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/models)**, which are then parsed into an in-memory dictionary.</br></br>
Now, let's learn **everything we can about our device's interfaces**.

In [77]:
learned_interfaces = device.learn("interface")
print(f"\n\n\n\n\n\n\n{json.dumps(learned_interfaces.to_dict(),indent=4)}")



2025-11-12 11:01:52,177: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'show vrf all detail' +++
show vrf all detail
Wed Nov 12 11:49:17.465 UTC
RP/0/RP0/CPU0:ios#

2025-11-12 11:01:52,841: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'show interface detail' +++
show interface detail
Wed Nov 12 11:49:18.070 UTC
Loopback100 is up, line protocol is up 
  Interface state transitions: 1
  Hardware is Loopback interface(s)
  Description: ***TEST LOOPBACK****
  Internet address is 1.1.1.100/32
  MTU 1500 bytes, BW 0 Kbit
     reliability Unknown, txload Unknown, rxload Unknown
  Encapsulation Loopback,  loopback not set,
  Last link flapped 00:46:23
  Last input Unknown, output Unknown
  Last clearing of "show interface" counters Unknown
  Input/output data rate is disabled.

Loopback200 is up, line protocol is up 
  Interface state transitions: 1
  Hardware is Loopback interface(s)
  Internet address is Unknown
  MTU 1500 bytes, BW 0 Kbit
     reliability Unk

🔌 This is **way more** than a simple `show ip interface brief`, right?!

⚡️ But now, let's create a new interface. We can do it either "manually":

In [78]:
device.configure('''
    interface Loopback401
    description "TechTilesDemo01"
''')


2025-11-12 11:03:21,362: %UNICON-INFO: +++ sandbox1 with via 'cli': configure +++
configure terminal
Wed Nov 12 11:50:46.769 UTC
RP/0/RP0/CPU0:ios(config)#
RP/0/RP0/CPU0:ios(config)#    interface Loopback401
RP/0/RP0/CPU0:ios(config-if)#    description "TechTilesDemo01"
RP/0/RP0/CPU0:ios(config-if)#commit
Wed Nov 12 11:50:47.815 UTC
RP/0/RP0/CPU0:ios(config-if)#end
RP/0/RP0/CPU0:ios#


'\r\n\rRP/0/RP0/CPU0:ios(config)    interface Loopback401\r\n\rRP/0/RP0/CPU0:ios(config-if)    description "TechTilesDemo01"\r\n\rRP/0/RP0/CPU0:ios(config-if)commit\r\n\rWed Nov 12 11:50:47.815 UTC\r\nRP/0/RP0/CPU0:ios(config-if)'

🤖 Or, we can use [pyATS built-in config classes](https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/models) to let it figure out the specifics for our device. We just provide the essential information!

In [79]:
from genie.conf.base import Interface

new_iosxr_interface = Interface(name="Loopback503", device=device)
new_iosxr_interface.description = "TechTilesDemo02"
rendered_config = new_iosxr_interface.build_config()
print(rendered_config)


2025-11-12 11:06:03,063: %UNICON-INFO: +++ sandbox1 with via 'cli': configure +++
configure terminal
Wed Nov 12 11:53:28.444 UTC
RP/0/RP0/CPU0:ios(config)#interface Loopback503
RP/0/RP0/CPU0:ios(config-if)# description TechTilesDemo02
RP/0/RP0/CPU0:ios(config-if)# exit
RP/0/RP0/CPU0:ios(config)#commit
Wed Nov 12 11:53:29.641 UTC
RP/0/RP0/CPU0:ios(config)#end
RP/0/RP0/CPU0:ios#
None


✨ **Lovely**. Now, let's **learn** again the interfaces and check **what changed since the last time that we did it**.

In [80]:
from genie.utils.diff import Diff

learned_interfaces_afterwards = device.learn("interface")
diff = Diff(learned_interfaces.info, learned_interfaces_afterwards.info)
diff.findDiff()
print(diff)



2025-11-12 11:07:20,594: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'show vrf all detail' +++
show vrf all detail
Wed Nov 12 11:54:45.796 UTC
RP/0/RP0/CPU0:ios#

2025-11-12 11:07:21,155: %UNICON-INFO: +++ sandbox1 with via 'cli': executing command 'show interface detail' +++
show interface detail
Wed Nov 12 11:54:46.478 UTC
Loopback100 is up, line protocol is up 
  Interface state transitions: 1
  Hardware is Loopback interface(s)
  Description: ***TEST LOOPBACK****
  Internet address is 1.1.1.100/32
  MTU 1500 bytes, BW 0 Kbit
     reliability Unknown, txload Unknown, rxload Unknown
  Encapsulation Loopback,  loopback not set,
  Last link flapped 00:51:52
  Last input Unknown, output Unknown
  Last clearing of "show interface" counters Unknown
  Input/output data rate is disabled.

Loopback200 is up, line protocol is up 
  Interface state transitions: 1
  Hardware is Loopback interface(s)
  Internet address is Unknown
  MTU 1500 bytes, BW 0 Kbit
     reliability Unk

➕ Noice! So we see in this diff that two new interfaces were created.

🔥 But now, we want to wipe clean our changes. We can easily retrieve the configurations applied by pyATS and remove them!

In [83]:
iface = device.interfaces.get("Loopback503")
rollback_config = iface.build_unconfig()
print(rollback_config)


2025-11-12 11:11:12,956: %UNICON-INFO: +++ sandbox1 with via 'cli': configure +++
configure terminal
Wed Nov 12 11:58:38.306 UTC
RP/0/RP0/CPU0:ios(config)#no interface Loopback503
RP/0/RP0/CPU0:ios(config)#commit
Wed Nov 12 11:58:38.893 UTC
RP/0/RP0/CPU0:ios(config)#end
RP/0/RP0/CPU0:ios#
None


🧹 Stunning. The other interface, however, needs to be rolled back manually as it was not created using the pyATS models, and our device instance doesn't keep track of it for rollback.

In [85]:
device.configure('''
    no interface Loopback401
''')


2025-11-12 11:12:32,126: %UNICON-INFO: +++ sandbox1 with via 'cli': configure +++
configure terminal
Wed Nov 12 11:59:57.469 UTC
RP/0/RP0/CPU0:ios(config)#
RP/0/RP0/CPU0:ios(config)#    no interface Loopback401
RP/0/RP0/CPU0:ios(config)#commit
Wed Nov 12 11:59:58.511 UTC
RP/0/RP0/CPU0:ios(config)#end
RP/0/RP0/CPU0:ios#


'\r\n\rRP/0/RP0/CPU0:ios(config)    no interface Loopback401\r\n\rRP/0/RP0/CPU0:ios(config)commit\r\n\rWed Nov 12 11:59:58.511 UTC\r\nRP/0/RP0/CPU0:ios(config)'